In [1]:
import json

API_KEY_FILE = 'api_key.txt'
REGION = 'euw1'
BASE_URL = 'api.riotgames.com'

QUEUE_TYPE = 'RANKED_SOLO_5x5'

# Ligas de las que extraer jugadores
PIPELINE = [
    {
        'url': '/lol/league/v4/challengerleagues/by-queue/{queue}',
        'name': 'CHALLENGER',
        'completed': False,
        'params': {
            'queue': QUEUE_TYPE,
        }
    },
    {
        'url': '/lol/league/v4/grandmasterleagues/by-queue/{queue}',
        'name': 'GRANDMASTER',
        'completed': False,
        'params': {
            'queue': QUEUE_TYPE,
        }
        
    },
    {
        'url': '/lol/league/v4/masterleagues/by-queue/{queue}',
        'name': 'MASTER',
        'completed': False,
        'params': {
            'queue': QUEUE_TYPE,
        }
    },
    {
        'url': '/lol/league/v4/entries/{queue}/{tier}/{division}',
        'completed': False,
        'params': {
            'queue': QUEUE_TYPE,
            'tier': 'DIAMOND',
            'division': 'I'
        },
        'page': 1
    },
    {
        'url': '/lol/league/v4/entries/{queue}/{tier}/{division}',
        'completed': False,
        'params': {
            'queue': QUEUE_TYPE,
            'tier': 'DIAMOND',
            'division': 'II',
        },
        'page': 1
    },
    {
        'url': '/lol/league/v4/entries/{queue}/{tier}/{division}',
        'completed': False,
        'params': {
            'queue': QUEUE_TYPE,
            'tier': 'DIAMOND',
            'division': 'III'
        },
        'page': 1
    },
    {
        'url': '/lol/league/v4/entries/{queue}/{tier}/{division}',
        'completed': False,
        'params': {
            'queue': QUEUE_TYPE,
            'tier': 'DIAMOND',
            'division': 'IV'
        },
        'page': 1
    }
]

with open(API_KEY_FILE) as f:
    api_key = f.read()
    
def build_url(endpoint):
    return f'https://{REGION}.{BASE_URL}{endpoint}'

def save_json(obj, file):
    with open(f'outputs/{file}', 'w+') as f:
        json.dump(obj, f, indent=4)

In [2]:
import requests, copy, time
import os, shutil, pickle

execution_cache_dir = 'current_execution'
leagues_state_file = f'{execution_cache_dir}/remaining_leagues.pkl'
players_file = f'{execution_cache_dir}/players.pkl'

headers = {
  'X-Riot-Token': api_key
}

os.makedirs(execution_cache_dir, exist_ok=True)
MIN_DELAY = 20/100 # esperar 20ms entre peticiones
delay = MIN_DELAY

# Leer un fichero del estado de la ejecución si existe o copiar PIPELINE
# para empezar desde 0
if os.path.exists(leagues_state_file) and os.stat(leagues_state_file).st_size != 0:
    with open(leagues_state_file, 'rb') as f:
        leagues_current_execution = pickle.load(f)
else:
    leagues_current_execution = copy.deepcopy(PIPELINE)

if os.path.exists(players_file) and os.stat(players_file).st_size != 0:
    with open(leagues_state_file, 'rb') as f:
        players = pickle.load(f)
else:
    players = []

# Filtrar las fases que ya estén completas
leagues_current_execution = list(filter(lambda league: not league['completed'], leagues_current_execution))

for league in leagues_current_execution:
    # Para las ligas paginadas, pedir en bucle hasta que no se devuelvan jugadores
    # Si alguna petición falla, aumentar delay
    # Si va bien, disminuir el delay hasta el inicial
    # Al terninar cada fase o página, guardar el estado
    if 'page' in league:
        params = league['params']
        while True:
            url = build_url(league['url'].format(**league['params']))
            response = requests.get(url, headers=headers, params={'page': league['page']})
            time.sleep(delay)
            if response.status_code == 200:
                league['page'] += 1
                players_temp = response.json()
                if not players_temp:
                    break
                players.extend(players_temp)
                print(f"{params['tier']} {params['division']} ({league['page']}): len: {len(players_temp)} / total: {len(players)}")
                delay = max(delay/2, MIN_DELAY)
            else:
                delay *= 2
            with open(leagues_state_file, 'wb') as f:
                pickle.dump(leagues_current_execution, f)
            with open(players_file, 'wb') as f:
                pickle.dump(players, f)
    else:
        url = build_url(league['url'].format(**league['params']))
        response = requests.get(url, headers=headers)
        time.sleep(delay)
        if response.status_code == 200:
            players_temp = response.json()['entries']
            players.extend(players_temp)
            print(f"{league['name']}: len: {len(players_temp)} / total: {len(players)}")
            delay = max(delay/2, MIN_DELAY)
        else:
            delay *= 2
    league['completed'] = True
    with open(leagues_state_file, 'wb') as f:
        pickle.dump(leagues_current_execution, f)
    with open(players_file, 'wb') as f:
        pickle.dump(players, f)

CHALLENGER: len: 300 / total: 300
GRANDMASTER: len: 700 / total: 1000
MASTER: len: 5066 / total: 6066
DIAMOND I (2): len: 205 / total: 6271
DIAMOND I (3): len: 205 / total: 6476
DIAMOND I (4): len: 205 / total: 6681
DIAMOND I (5): len: 205 / total: 6886
DIAMOND I (6): len: 205 / total: 7091
DIAMOND I (7): len: 205 / total: 7296
DIAMOND I (8): len: 205 / total: 7501
DIAMOND I (9): len: 205 / total: 7706
DIAMOND I (10): len: 205 / total: 7911
DIAMOND I (11): len: 205 / total: 8116
DIAMOND I (12): len: 205 / total: 8321
DIAMOND I (13): len: 205 / total: 8526
DIAMOND I (14): len: 205 / total: 8731
DIAMOND I (15): len: 205 / total: 8936
DIAMOND I (16): len: 205 / total: 9141
DIAMOND I (17): len: 205 / total: 9346
DIAMOND I (18): len: 205 / total: 9551
DIAMOND I (19): len: 205 / total: 9756
DIAMOND I (20): len: 205 / total: 9961
DIAMOND I (21): len: 205 / total: 10166
DIAMOND I (22): len: 205 / total: 10371
DIAMOND I (23): len: 205 / total: 10576
DIAMOND I (24): len: 205 / total: 10781
DIAMO

DIAMOND IV (57): len: 205 / total: 46686
DIAMOND IV (58): len: 205 / total: 46891
DIAMOND IV (59): len: 205 / total: 47096
DIAMOND IV (60): len: 205 / total: 47301
DIAMOND IV (61): len: 205 / total: 47506
DIAMOND IV (62): len: 205 / total: 47711
DIAMOND IV (63): len: 205 / total: 47916
DIAMOND IV (64): len: 205 / total: 48121
DIAMOND IV (65): len: 205 / total: 48326
DIAMOND IV (66): len: 205 / total: 48531
DIAMOND IV (67): len: 205 / total: 48736
DIAMOND IV (68): len: 205 / total: 48941
DIAMOND IV (69): len: 205 / total: 49146
DIAMOND IV (70): len: 205 / total: 49351
DIAMOND IV (71): len: 205 / total: 49556
DIAMOND IV (72): len: 205 / total: 49761
DIAMOND IV (73): len: 205 / total: 49966
DIAMOND IV (74): len: 205 / total: 50171
DIAMOND IV (75): len: 205 / total: 50376
DIAMOND IV (76): len: 205 / total: 50581
DIAMOND IV (77): len: 205 / total: 50786
DIAMOND IV (78): len: 205 / total: 50991
DIAMOND IV (79): len: 205 / total: 51196
DIAMOND IV (80): len: 205 / total: 51401
DIAMOND IV (81):

In [3]:
# Guardar JSON con los jugadores
fields = {'summonerId', 'summonerName'}
summoners = [{k: player[k] for k in fields} for player in players]
save_json(summoners, 'players.json')

In [4]:
# Limpiar estados
shutil.rmtree(execution_cache_dir)